# Задание 1.3
Напишите запрос,чтобы перечислить всех покупателей выбранного магазина с указанием их полных имен, электронной почты и номера телефона.

(Представлены два способа решения)

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local").getOrCreate()


API

In [14]:
from pyspark.sql.functions import concat_ws

df_customers = spark.read.csv("CVS\\customers.csv", inferSchema=True, header=True, sep=",")
df_orders = spark.read.csv("CVS\\orders.csv", inferSchema=True, header=True, sep=",")
df_stores = spark.read.csv("CVS\\stores.csv", inferSchema=True, header=True, sep=",")

df_res = df_customers.join(df_orders, 
                           df_orders["customer_id"] == df_customers["customer_id"])

df_res = df_res.join(df_stores, 
                     df_stores["store_id"] == df_orders["store_id"]
                     ).where(df_stores["store_name"] == 'Rowlett Bikes')

df_res = df_res.select(
    concat_ws(" ",df_customers["first_name"], 
              df_customers["last_name"]).alias("customer"),
              df_customers["email"], df_customers["phone"])

df_res.dropDuplicates().show()

+---------------+--------------------+--------------+
|       customer|               email|         phone|
+---------------+--------------------+--------------+
| Ivelisse Nixon|ivelisse.nixon@ao...|(281) 941-4933|
|   Omer Estrada|omer.estrada@gmai...|          NULL|
|  Nestor Haynes|nestor.haynes@msn...|(281) 969-4579|
|     Dagny Owen|dagny.owen@hotmai...|          NULL|
| Mozelle Carter|mozelle.carter@ao...|(281) 489-9656|
|  Andria Rivers|andria.rivers@aol...|          NULL|
| Minerva Decker|minerva.decker@ya...|(281) 271-6390|
|Dorthey Jackson|dorthey.jackson@m...|(281) 926-8010|
|Pinkie Kirkland|pinkie.kirkland@y...|          NULL|
|Shaunda Barnett|shaunda.barnett@g...|          NULL|
|     Launa Hull|launa.hull@yahoo.com|          NULL|
|  Aleta Shepard|aleta.shepard@aol...|          NULL|
|     Gena Owens|gena.owens@gmail.com|          NULL|
| Thalia Dillard|thalia.dillard@ya...|          NULL|
|   Liliana Kerr|liliana.kerr@yaho...|          NULL|
|    Londa Gould| londa.goul

spark.sql

In [16]:
df_customers.createOrReplaceTempView("customers")
df_stores.createOrReplaceTempView("stores")
df_orders.createOrReplaceTempView("orders")

spark.sql("""
          SELECT CONCAT_WS(' ',customers.first_name, customers.last_name) as customer, customers.email, customers.phone
            FROM customers
            JOIN orders ON orders.customer_id = customers.customer_id
            JOIN stores ON stores.store_id = orders.store_id
            WHERE stores.store_name = 'Rowlett Bikes'
          """).dropDuplicates().show()

+---------------+--------------------+--------------+
|       customer|               email|         phone|
+---------------+--------------------+--------------+
| Ivelisse Nixon|ivelisse.nixon@ao...|(281) 941-4933|
|   Omer Estrada|omer.estrada@gmai...|          NULL|
|  Nestor Haynes|nestor.haynes@msn...|(281) 969-4579|
|     Dagny Owen|dagny.owen@hotmai...|          NULL|
| Mozelle Carter|mozelle.carter@ao...|(281) 489-9656|
|  Andria Rivers|andria.rivers@aol...|          NULL|
| Minerva Decker|minerva.decker@ya...|(281) 271-6390|
|Dorthey Jackson|dorthey.jackson@m...|(281) 926-8010|
|Pinkie Kirkland|pinkie.kirkland@y...|          NULL|
|Shaunda Barnett|shaunda.barnett@g...|          NULL|
|     Launa Hull|launa.hull@yahoo.com|          NULL|
|  Aleta Shepard|aleta.shepard@aol...|          NULL|
|     Gena Owens|gena.owens@gmail.com|          NULL|
| Thalia Dillard|thalia.dillard@ya...|          NULL|
|   Liliana Kerr|liliana.kerr@yaho...|          NULL|
|    Londa Gould| londa.goul